In [73]:
import pandas as pd
df = pd.read_csv('../csv_data/Mitsubishi/Mitsubishi_2.csv')

df

,TEXT,X1,Y1,X2,Y2,block_no,line_no
0,______________________________________________...,50,445,560,459,0,0
1,Item,50,463,70,477,1,0
2,Material,115,463,152,477,1,1
3,Description,178,463,230,477,1,2
4,Qty,189,475,205,489,2,0
...,...,...,...,...,...,...,...
292,Tullow,123,283,153,297,45,2
293,CARLOW,50,295,93,309,45,3
294,R93EV90,50,307,94,321,45,4
295,IRELAND,50,319,92,333,45,5


In [74]:
def find_text(temp_df, text,case=False):
    location = temp_df[temp_df['TEXT'].str.contains(text,case=case, na=False)].index.tolist()
    return location

def find_next_word(temp_df, text, nearest_text, case=False):
    pos = find_text(temp_df,text,case=case)
    pos = [item +1 for item in pos]
#     print(pos)
    new_pos = list()
    for item in pos:
#         temp = df.loc[]
        temp_value= temp_df.loc[item]['TEXT']
#         print(temp_value.lower())
       
        if temp_value.lower().__contains__(nearest_text.lower()):
            new_pos.append(item-1)
        
    return new_pos

In [75]:
pos =  find_text(df,'Invoice')

In [77]:
invoice_no = df[pos[0]+3:pos[0]+4]['TEXT'].values[0]
invoice_no

'51856019'

In [78]:
pos_date = find_text(df,'Date')
invoice_date = df[pos_date[0]+2:pos_date[0]+3]['TEXT'].values[0]
invoice_date

'02.04.2020'

In [83]:
find_text(df, 'branch')


[118, 131, 161]

[281]

In [88]:
table_start = find_text(df,'Item')
table_end = find_text(df,'______________________________________________')
if len(table_end) < 3:
    table_end = find_text(df,'commodity')

table_data= df[table_start[0]:table_end[-1]+1]
table_data

,TEXT,X1,Y1,X2,Y2,block_no,line_no
1,Item,50,463,70,477,1,0
2,Material,115,463,152,477,1,1
3,Description,178,463,230,477,1,2
4,Qty,189,475,205,489,2,0
5,Price/piece,241,475,291,489,2,1
...,...,...,...,...,...,...,...
86,EUR,312,691,332,705,19,1
87,1,353,691,359,705,19,1
88,PC,365,691,378,705,19,1
89,114.30,518,691,553,705,19,2


In [89]:
from pprint import pprint
msg = dict()
result = list()
for index, row in table_data.iterrows():
    
    if  (40 < row['X1'] < 87) and (row['TEXT'].isdigit()) : 
#         print(row['TEXT'])
        msg = dict()
        msg['item'] = row['TEXT']
        msg['disc'] = 0
        msg['desc_not'] = True
        print(row['TEXT'])
        
    if ('item' in msg) and (110 < row['X1'] < 142):
        msg['part'] = row['TEXT']
        print(row['TEXT'])
        msg['desc'] = ''
        
    if 'part' in msg and (173 < row['X1'] < 400) and msg['desc_not']:
        msg['desc'] = msg['desc'] + " " + row['TEXT']
        
        
        
    if row['TEXT'] =='PC' and not 'qty' in msg  :
        print(table_data.iloc[index-2]['TEXT'])
        msg['qty'] = table_data.iloc[index-2]['TEXT']
        msg['desc_not'] = False
        
        
    if row['TEXT'].__contains__('Discount'):
        msg['disc'] = table_data.iloc[index]['TEXT'].replace('-','')
        
    
    if row['TEXT'].__contains__("Commodity"):
        msg['total'] = table_data.iloc[index-2]['TEXT']
        print(msg)
        
    
# #         print(row['TEXT'])
#         msg['part'] = row['TEXT']
#         msg['desc'] = ''
   
#     if ('desc' in msg) and (180 < row['X1'] < 360):
# #         print(row['TEXT'])
#         msg['desc'] = msg['desc'] +" "+ row['TEXT']
#         msg['unit'] = ''
   
#     if ('unit' in msg) and (400  < row['X1'] < 460):
# #         print(row['TEXT'])
#         msg['unit'] = row['TEXT']
#         msg['disc'] = 0
#         msg['total'] = 0 
        
#     if 'disc' in msg and 470  < row['X1'] < 500:
#         msg['disc'] = row['TEXT']
#         msg['total'] = 0
        
#     if 'total' in msg and 520 < row['X1'] < 560:
        
#         msg['total'] = row['TEXT']
#         pprint(msg)
#         result.append(msg)

000010
313825
1
{'item': '000010', 'disc': '21.57', 'desc_not': False, 'part': '313825', 'desc': ' GT2507-WTSD GOT2000;7"wideTFT;DC;WVGA;65k colors;RS-422/485;RS-232;USB;Eth;SD;Silver 1 PC', 'qty': '1', 'total': '1,050.96'}
000020
316588
1
{'item': '000020', 'disc': '44.44', 'desc_not': False, 'part': '316588', 'desc': ' FR-E740-170SC-ENE Inverter; Rated Power: 7,5kW; 3x380-480V; Rated Current: 17A@50°C; IP20 1 PC', 'qty': '1', 'total': '554.94'}
000030
280505
1
{'item': '000030', 'disc': 0, 'desc_not': False, 'part': '280505', 'desc': ' FX5-16EX/ES FX5;I/O Extension;16 Inputs 1 PC', 'qty': '1', 'total': '114.30'}


,TEXT,X1,Y1,X2,Y2,block_no,line_no
1,Item,50,463,70,477,1,0
2,Material,115,463,152,477,1,1
3,Description,178,463,230,477,1,2
4,Qty,189,475,205,489,2,0
5,Price/piece,241,475,291,489,2,1
6,Value/EUR,506,475,555,489,2,2
7,______________________________________________...,50,481,560,495,2,3
8,000020,50,499,87,513,3,0
9,316585,115,499,152,513,3,1
10,FR-E740-060SC-ENE,178,499,277,513,3,2


[0, 7, 46]